# 02 Panel Data

![img](https://cdn.dribbble.com/users/502247/screenshots/2446485/more-_-more_2_.gif)

## Importing the Packages Needed

In [ ]:
import pandas as pd

from linearmodels import PanelOLS

import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [ ]:
df = pd.read_csv('data/Hyp_panelperformance.csv')

## Inspecting the Data

In [ ]:
df.head() # gives you back a view of the first 5 rows

In [ ]:
df.drop('panel_id', axis=1).describe().T # checking out the descriptive statistics of our numerical variables

In [ ]:
df.shape # looking at the amount of rows and columns

In [ ]:
df.isna().sum() # checking if there are any missing values

## 1. Regression Examples

The Goal of a Regression is to search for associations between one or many variables and another. For example, determining the price of a house (what we want to predict) might only be possible with additional information (what will help us make a prediction) such as # of bathrooms, # of bedrooms, # garage, etc...

A regression is a type of linear model with which we can quantify the relationships in our data and, at the same time, determine how reliable such relationship is. A linear model usually looks as follows,

$y = a*x + b$

- $a$ - is the slope of the line
- $b$ - y intercept is where the line crosses the y-axis
- $y$ - is what we are trying to predict
- $x$ - is what we are using to predict

We are interested in finding the optimal values or $a$ and $b$ which are also called parameters. 

## OLS Regression Example 1

In [ ]:
y = df['performance']

In [ ]:
X = sm.add_constant(df[['clientactivity', 'overtime', 'seniorityrank', 'collaboration', 'managerrelationship', 'slackactivity', 'externalvisibility']])

In [ ]:
# run a regression on our main metric using all variables in our dataset as the independent variables
model = sm.OLS(y, X).fit()

# print the summary
print(model.summary())

## OLS Regression Example 2

In [ ]:
X_2 = sm.add_constant(df.drop(['performance', 'panel_id', 'year', 'teamno5'], axis=1))

In [ ]:
# run a regression on our main metric using all variables in our dataset as the independent variables
model2 = sm.OLS(y, X_2).fit()

# print the summary
print(model2.summary())

## Panel Data

Panel data is a type of data that comes with two main characteristics that are, is time series and cross-sectional data.

> "Panel data, also known as longitudinal data or cross-sectional time series data in some special cases, is data that is derived from a (usually small) number of observations over time on a (usually large) number of cross-sectional units like individuals, households, firms, or governments." by Mike Moffatt at [thoughtco](https://www.thoughtco.com/panel-data-definition-in-economic-research-1147034)

This is the main formula,  
$Y_{ijt} = α + βX_{ijt} + ϕ_i + θ_j + μ_t + ε_{ijt}$

But this is another way to look at it,

$Y_{👩 👯‍ ⏳} = α + β🏄_{👩 👯‍ ⏳} + 🎬_{👩} + 🎬_{👯‍} + 🎬_{⏳} + ε_{👩 👯‍ ⏳}$


- $i$ or 👩🏽‍ = individual  
- $j$ or 👯‍ = team  
- $t$ or ⏳ = year  
- $Y_{ijt}$ or $Y_{👩 👯‍ ⏳}$ = observed performance of individual i working in team j in year t  
- $α$ = constant  
- $X$ or 🏄🏽‍ = employees’ actions (e.g. client activity, slack activity, external visibility…)  
- $ϕ_i$ = effect of individual  
- $θ_j$ = effect of team  
- $μ_t$ = effect of year  
- $ε_{ijt}$ = random noise  

Since we will need a multi-index, we will first convert our year variable into a datetime format using the function `pd.to_datetime` from pandas.

In [ ]:
df['year'] = pd.to_datetime(df['year'], format='%Y').dt.year

We will then use the `.set_index()` method to assign our variables (in the order we need them to be), to the index of our dataframe.

In [ ]:
new_df = df.set_index(['panel_id', 'year']).copy()

In [ ]:
print(new_df.shape)

In [ ]:
new_df.head()

## Panel Data Regression Example 1

In [ ]:
independent_vars = ['clientactivity', 'overtime', 'seniorityrank', 'collaboration',
                    'managerrelationship', 'slackactivity', 'externalvisibility', 
                    'teamno1', 'teamno2', 'teamno3', 'teamno4', 'teamno6',
                    'teamno7', 'teamno8', 'teamno9', 'teamno10']

In [ ]:
model3 = PanelOLS(dependent=new_df['performance'], exog=sm.add_constant(new_df[independent_vars]), 
                  entity_effects=True, time_effects=True)
print(model3.fit())

## Panel Data Regression Example 2

In [ ]:
model4 = PanelOLS.from_formula("""performance ~ 1 + clientactivity + overtime + seniorityrank + collaboration + 
                                                    managerrelationship + slackactivity + externalvisibility + teamno1 + 
                                                    teamno2 + teamno3 + teamno4 + teamno6 + teamno7 + teamno8 + teamno9 + 
                                                    teamno10 + EntityEffects + TimeEffects""", data=new_df).fit()
print(model4)